In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/ml_pr.zip"  # Change this to your actual path
extract_path = "/content/dataset2"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [ ]:
import os

print("Folders inside dataset:")
print(os.listdir("/content/dataset2"))


Folders inside dataset:
['food_ml']


In [ ]:
!pip install tensorflow

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/dataset2/food_ml/training"
val_dir = "/content/dataset2/food_ml/validation"

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Adjust based on your model
    batch_size=32,
    class_mode='categorical'  # Use 'binary' if only two classes
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 577.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.4 MB/s eta 0:00:00
Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.


In [ ]:
import tensorflow as tf
print(tf.__version__)  # ✅ Check TensorFlow version


2.19.0


In [ ]:
# Load images and extract the number of classes
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Correctly define num_classes from train_generator
num_classes = len(train_generator.class_indices)  # Get total classes
print(f"Number of food categories: {num_classes}")


Found 9866 images belonging to 11 classes.
Number of food categories: 11


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# ✅ Ensure eager execution is enabled
tf.compat.v1.enable_eager_execution()

# 📂 Define dataset paths
train_dir = "/content/dataset2/food_ml/training"
val_dir = "/content/dataset2/food_ml/validation"

# ✅ Data Augmentation to prevent overfitting
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for validation data

# ✅ Load training and validation images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# ✅ Number of classes detected automatically
num_classes = len(train_generator.class_indices)

# ✅ Define model architecture using MobileNetV2
def create_model():
    base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Freeze base model layers

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])

    # ✅ Define optimizer with a learning rate
    learning_rate = 0.0001
    optimizer = optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ✅ Checkpoint directory
checkpoint_path = "model_checkpoint.h5"

# ✅ Load existing model if available, else create a new one
if os.path.exists(checkpoint_path):
    print("🔄 Previous checkpoint found. Loading model...")
    model = tf.keras.models.load_model(checkpoint_path, compile=False)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
else:
    print("🆕 No checkpoint found. Creating new model...")
    model = create_model()

# ✅ Checkpoint callback to save model every phase
checkpoint_callback = ModelCheckpoint(checkpoint_path, save_best_only=True, verbose=1)

# ✅ Reduce learning rate if validation loss doesn't improve
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# ✅ Split training into 4 phases (5 epochs each) to avoid Colab disconnection
def train_in_phases(start_epoch, end_epoch):
    try:
        print(f"\n🚀 **Starting Phase {start_epoch}-{end_epoch}**\n")
        history = model.fit(
            train_generator,
            epochs=end_epoch,
            initial_epoch=start_epoch - 1,
            validation_data=val_generator,
            callbacks=[checkpoint_callback, reduce_lr]
        )
        print(f"✅ **Phase {start_epoch}-{end_epoch} completed!**")
    except KeyboardInterrupt:
        print("⏸ Training stopped manually. Model saved at last checkpoint.")
    except Exception as e:
        print(f"⚠️ Error occurred: {e}. Training will resume next time.")

# ✅ Train in 4 phases of 5 epochs each
train_in_phases(1, 5)
train_in_phases(6, 10)
train_in_phases(11, 15)
train_in_phases(16, 20)

print("\n🎉 **Training Complete! Model is saved as model_checkpoint.h5**")


Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
🆕 No checkpoint found. Creating new model...


<ipython-input-7-40056b7a0d46>:49: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

🚀 **Starting Phase 1-5**



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 981ms/step - accuracy: 0.3361 - loss: 6.1643
Epoch 1: val_loss improved from inf to 4.54635, saving model to model_checkpoint.h5


...

309/309 ━━━━━━━━━━━━━━━━━━━━ 403s 1s/step - accuracy: 0.8036 - loss: 1.0825 - val_accuracy: 0.7778 - val_loss: 1.1290 - learning_rate: 1.0000e-04
Epoch 17/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 968ms/step - accuracy: 0.7996 - loss: 1.0498
Epoch 17: val_loss improved from 1.12905 to 1.08219, saving model to model_checkpoint.h5


309/309 ━━━━━━━━━━━━━━━━━━━━ 401s 1s/step - accuracy: 0.7996 - loss: 1.0498 - val_accuracy: 0.7810 - val_loss: 1.0822 - learning_rate: 1.0000e-04
Epoch 18/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 980ms/step - accuracy: 0.8081 - loss: 0.9856
Epoch 18: val_loss improved from 1.08219 to 1.04865, saving model to model_checkpoint.h5


309/309 ━━━━━━━━━━━━━━━━━━━━ 404s 1s/step - accuracy: 0.8081 - loss: 0.9856 - val_accuracy: 0.7878 - val_loss: 1.0486 - learning_rate: 1.0000e-04
Epoch 19/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 986ms/step - accuracy: 0.8132 - loss: 0.9429
Epoch 19: val_loss improved from 1.04865 to 1.04006, saving model to model_checkpoint.h5


309/309 ━━━━━━━━━━━━━━━━━━━━ 406s 1s/step - accuracy: 0.8132 - loss: 0.9429 - val_accuracy: 0.7726 - val_loss: 1.0401 - learning_rate: 1.0000e-04
Epoch 20/20
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 976ms/step - accuracy: 0.8130 - loss: 0.9182
Epoch 20: val_loss improved from 1.04006 to 0.99161, saving model to model_checkpoint.h5


309/309 ━━━━━━━━━━━━━━━━━━━━ 401s 1s/step - accuracy: 0.8130 - loss: 0.9182 - val_accuracy: 0.7843 - val_loss: 0.9916 - learning_rate: 1.0000e-04
✅ **Phase 16-20 completed!**

🎉 **Training Complete! Model is saved as model_checkpoint.h5**


In [ ]:
print(train_generator.class_indices)


{'Bread': 0, 'Dairy product': 1, 'Dessert': 2, 'Egg': 3, 'Fried food': 4, 'Meat': 5, 'Noodles-Pasta': 6, 'Rice': 7, 'Seafood': 8, 'Soup': 9, 'Vegetable-Fruit': 10}


In [ ]:
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy:.2f}, Loss: {loss:.2f}")


108/108 ━━━━━━━━━━━━━━━━━━━━ 100s 927ms/step - accuracy: 0.7920 - loss: 0.9762
Validation Accuracy: 0.78, Loss: 0.99


In [ ]:
!pip install tensorflow --upgrade --quiet
import tensorflow as tf
print(tf.__version__)


2.19.0


In [ ]:
print(train_generator.class_indices)
print(val_generator.class_indices)


{'Bread': 0, 'Dairy product': 1, 'Dessert': 2, 'Egg': 3, 'Fried food': 4, 'Meat': 5, 'Noodles-Pasta': 6, 'Rice': 7, 'Seafood': 8, 'Soup': 9, 'Vegetable-Fruit': 10}
{'Bread': 0, 'Dairy product': 1, 'Dessert': 2, 'Egg': 3, 'Fried food': 4, 'Meat': 5, 'Noodles-Pasta': 6, 'Rice': 7, 'Seafood': 8, 'Soup': 9, 'Vegetable-Fruit': 10}


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# ✅ Image ka path
img_path = "/content/39.jpg"

# ✅ Image preprocess karo
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# ✅ Prediction lo
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)[0]

# ✅ Class Mapping
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}  # Reverse Mapping

# ✅ Predicted Class Output
predicted_label = class_labels.get(predicted_class, "Unknown")
print(f"Predicted Class Index: {predicted_class}")
print(f"Predicted Class Name: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted Class Index: 0
Predicted Class Name: Bread


In [ ]:
from google.colab import drive
import tensorflow as tf

# Mount Google Drive
drive.mount('/content/drive')

# Save model
model.save('/content/drive/MyDrive/my_model.h5')  # Change the path as needed


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files

# Download the model file
files.download('/content/drive/MyDrive/my_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json

# Save class indices as JSON
with open('/content/drive/MyDrive/class_indices.json', 'w') as f:
    json.dump(train_generator.class_indices, f)

print("Class indices saved successfully!")


Class indices saved successfully!
